janomeについて

https://note.nkmk.me/python-janome-tutorial/

globについて

https://techacademy.jp/magazine/18928

In [1]:
from janome.tokenizer import Tokenizer
import re
import glob##パス名を取得

In [2]:
##ほんとは　序章部分も回すべき
no_zero = glob.glob("アルマーク/序章/*") 
no_zero

['アルマーク/序章/N9407FU-5.txt',
 'アルマーク/序章/N9407FU-4.txt',
 'アルマーク/序章/N9407FU-3.txt',
 'アルマーク/序章/N9407FU-2.txt',
 'アルマーク/序章/N9407FU-1.txt']

In [3]:
## transer用
no_zero =  ["Transer/N7235FU-8.txt"]

In [4]:
lines = open(no_zero[4],"r",encoding="utf-8")

#\n を削除した文章化(\u3000 は全角空白？) はしない
lines = lines.read()##re.sub(r"[akira]","",text.read())
        
t = Tokenizer()
streams = [print(token) for token in t.tokenize(lines)]

よろしく	副詞,一般,*,*,*,*,よろしく,ヨロシク,ヨロシク
お願い	名詞,サ変接続,*,*,*,*,お願い,オネガイ,オネガイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。

	記号,空白,*,*,*,*,
,*,*
********************************************	名詞,サ変接続,*,*,*,*,********************************************,*,*

	記号,空白,*,*,*,*,
,*,*
北	名詞,一般,*,*,*,*,北,キタ,キタ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
傭兵	名詞,一般,*,*,*,*,傭兵,ヨウヘイ,ヨーヘイ


	記号,空白,*,*,*,*,

,*,*
　	記号,空白,*,*,*,*,　,　,
 	記号,空白,*,*,*,*, ,*,*
アル	名詞,固有名詞,人名,名,*,*,アル,アル,アル
マーク	名詞,サ変接続,*,*,*,*,マーク,マーク,マーク
の	助詞,連体化,*,*,*,*,の,ノ,ノ
ノルク	名詞,一般,*,*,*,*,ノルク,*,*
魔法	名詞,一般,*,*,*,*,魔法,マホウ,マホー
学院	名詞,一般,*,*,*,*,学院,ガクイン,ガクイン
入学	名詞,サ変接続,*,*,*,*,入学,ニュウガク,ニューガク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、
二	名詞,数,*,*,*,*,二,ニ,ニ
年	名詞,接尾,助数詞,*,*,*,年,ネン,ネン
遅れ	動詞,自立,*,*,一段,連用形,遅れる,オクレ,オクレ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。



	記号,空白,*,*,*,*,


,*,*
　	記号,空白,*,*,*,*,　,　,
中原	名詞,固有名詞,人名,姓,*,*,中原,ナカハラ,ナカハラ
、	記号,読点,*,*,*,*,、,、,、
南方	名詞,一般,*,*,*,*,南方,ナンポウ,ナンポー
で	助詞,格助詞,一般,*,*,*,で,デ,デ
戦乱	名詞,一般,*,*

In [11]:
import collections
def parse_voc(text):
    ##助動詞 or 助詞 + 別の品詞 + 、。 が来るまで合体させる
    vocs = []
    voc = ""##vocを合体させる部分
    before_voc = ""##前回のvoc字ズラのまま
    before_nomal_voc = ""##前回のvoc言い切り型
    before_word_type = ""
    remove = ["助動詞","助詞"]
    split_norm = ["数","代名詞"]##名刺でも取り除く(分解する分類)                #\u3000は全角空白?
    split_symbol = ["、","。","(",")","「","」","　'","’","'",'"',"&","quot",";","“","『","』","[","]","「&"]##記号は含めないが、ここで上手に分解する  "がなぜか &quot; になる

    for token in t.tokenize(text):#[”。”]は無理矢理処理数を増やすため
        ##print(token)
        now_word_type = token.part_of_speech.split(",")[0]
        
        ##助詞・助動詞の処理 + 、。の処理 + 改行や空白(\u3000)の時の処理
        if now_word_type in remove or token.surface in split_symbol or token.part_of_speech.split(",")[1] == "空白":
            voc += before_nomal_voc##動詞を切りのいい形で代入
            if voc:##空語(助詞・助動詞で生まれる)を取り除く
                vocs.append(voc)
            voc = ''
            before_word_type = ""
            before_nomal_voc = ""
        
        ##前回と同じ品詞 = 分割ミス として一つ前の変換値を繋げる + 名詞-数字の連鎖を断つための処理(この時、前が接頭辞かを確認すべき)
        elif before_word_type == "接頭詞" or (before_word_type == now_word_type and token.part_of_speech.split(",")[1] not in split_norm):
            voc += before_voc
            ##初期化
            before_voc = token.surface ##今の単語の字ズラままを入れる
            before_nomal_voc = token.base_form ##今の単語を読み切りの形に
            before_word_type = now_word_type##品詞変更
        else:
            voc += before_nomal_voc
            ##空語(助詞・助動詞で生まれる)を取り除く 
            if voc:
                vocs.append(voc)##前回までの単語を追加
                voc=""
            
            ##初期化
            before_voc = token.surface ##今の単語の字ズラままを入れる
            before_nomal_voc = token.base_form ##今の単語を読み切りの形に
            before_word_type = now_word_type##品詞変更
            
    ##集めた単語を返す
    return vocs

#c = collections.Counter(vocs)
#c

In [14]:
def concat_voc(dir_name):
    file_names = glob.glob(dir_name+"/*")
    voc_list = []
    
    ##print(file_names)
    for file_name in file_names:
        with open(file_name,"r",encoding="utf-8") as f:
            file = f.read()
            ##print(file)
            voc_list += parse_voc(file)
        
    return voc_list

In [15]:
voc_list = concat_voc("アルマーク/序章")
c = collections.Counter(voc_list)
c

Counter({'旅': 12,
         '傭兵団': 3,
         '実戦': 1,
         '経験': 2,
         '積む': 2,
         'いる': 47,
         'いう': 2,
         'アルマーク': 57,
         'たった': 1,
         '９才': 4,
         '子供': 6,
         '北': 9,
         '土地': 1,
         '一人旅': 4,
         '想像': 1,
         '絶する': 1,
         '過酷': 4,
         'もの': 5,
         'しかも': 1,
         '季節': 5,
         '秋': 2,
         '過ぎる': 4,
         '冬': 6,
         'なる': 25,
         'する': 53,
         '飢え': 1,
         '寒い': 1,
         'さ': 5,
         '魔物': 1,
         'それ': 9,
         '心': 3,
         '荒ぶ': 1,
         '人間たち': 2,
         'あらゆる': 1,
         '敵': 3,
         '彼': 21,
         '何度': 2,
         '命': 3,
         '落としかける': 2,
         'その': 18,
         '度': 1,
         '子供離れ': 1,
         '剣技': 3,
         '強運': 1,
         '窮地': 1,
         '脱す': 1,
         'くる': 13,
         '時には': 2,
         '移動中': 2,
         '入れる': 6,
         'もらう': 3,
         '戦士': 5,
         '魔術師': 2,
         '行動': 1,
      

0

In [21]:
[print(token) for token in t.tokenize(u"ギル・バ正規軍")]

ギル・バ	名詞,一般,*,*,*,*,ギル・バ,*,*
正規	名詞,一般,*,*,*,*,正規,セイキ,セイキ
軍	名詞,接尾,一般,*,*,*,軍,グン,グン


[None, None, None]